In [28]:
import json
from gensim import models
import numpy as np
from typing import Callable
import pandas as pd
import spacy
import itertools
from comparaison_BATS import *
import sys
sys.path.append('..')
from clustering import k_medoides as k_med
import distance_wmd as wmd

In [8]:
with open('../data/docs.json', encoding = "utf8") as f:
    docs: list[str] = json.load(f)
print(f"Nombre de documents : {len(docs)}")

Nombre de documents : 9501


In [25]:
with open('../data/liste_lemmes.txt') as f:
    vocabulaire = f.readlines()

# Suppression des retours chariots sur chaque ligne
for i in range(len(vocabulaire)):
    vocabulaire[i] = vocabulaire[i].replace('\n', '')

print(f"Taille du vocabulaire : {len(vocabulaire)}")

Taille du vocabulaire : 7231


# Création et entraînement basique

## Création du modèle

- `vector_size` : taille de l'espace vectoriel de représentation des mots
- `windows` : fenêtre gauche / droite de contexte
- `min_count` : valeur fréquence absolue minimale que doit avoir un mot pour être inclus
- `workers` : nombre de threads utilisés pour l'entraînement
- `sg` : 1 si Skip-Gram, 0 si CBOW

In [10]:
w2vec: models.Word2Vec = models.Word2Vec(sentences = docs, vector_size = 300, window = 2, min_count = 1, workers = 6, sg = 0)

In [11]:
# espace vectoriel généré des mots
ev = w2vec.wv

## Enregistrement du modèle

In [15]:
w2vec.save("../data/w2vec.bin")

## Récupération du modèle

In [19]:
w2vec: models.Word2Vec = models.Word2Vec.load("../data/w2vec.bin")

In [20]:
# espace vectoriel généré des mots
ev = w2vec.wv

# Observation qualitative

## Proximité de divers termes entre eux

In [26]:
def mots_plus_proches(espace: models.KeyedVectors, mot: str, nbMots: int = 10, distance: str | Callable = 'cosine') -> list[str]:
    resultats = {}
    for v in vocabulaire:
        if v != mot:
            if distance == 'cosine':
                resultats[v] = np.dot(espace[v], espace[mot]) / (np.linalg.norm(espace[v]) * np.linalg.norm(espace[mot]))

    resultats = sorted(resultats.items(), key = lambda x: x[1], reverse = True)
    for i in range(nbMots):
        print(resultats[i])

In [27]:
mots_plus_proches(ev, 'dechet', 10)

('faire', 0.9997729)
('nature', 0.9997668)
('citoyen', 0.99976385)
('être', 0.9997621)
('france', 0.9997531)
('environnement', 0.9997397)
('biodiversite', 0.9997345)
('pouvoir', 0.9997297)
('pays', 0.99972695)
('inciter', 0.9997259)


## Analogies

In [8]:
def mot_plus_proche(espace: models.KeyedVectors, vecteurMot: np.ndarray, nbMots: int = 5, distance: str | Callable = 'cosine') -> list[str]:
    resultats = {}
    for v in vocabulaire:
        if distance == 'cosine':
            resultats[v] = np.dot(espace[v], vecteurMot) / (np.linalg.norm(espace[v]) * np.linalg.norm(vecteurMot))

    resultats = sorted(resultats.items(), key = lambda x: x[1], reverse = True)
    for i in range(nbMots):
        print(resultats[i])

In [9]:
variation = ev['pluie'] - ev['eau']
mot_plus_proche(ev, ev['dechet'] + variation)

('dechet', 0.96101624)
('selectif', 0.9606815)
('chauffer', 0.96038085)
('centre', 0.9601055)
('eduquer', 0.9599925)


## Observation quantitative (comparaison avec un modèle stable)

In [10]:
nlp = spacy.load("fr_core_news_sm")
traducteur = GoogleTranslator(source='en',target='fr')

récupération d'un modèle word2vec de référence

In [11]:
w2vecR: models.KeyedVectors = models.KeyedVectors.load_word2vec_format("../data/frWiki_no_phrase_no_postag_1000_skip_cut100.bin", binary=True, unicode_errors="ignore")
vocabulaireCommun = set(vocabulaire).intersection(w2vecR.index_to_key)

In [12]:
def get_position_proximite(modele: models.KeyedVectors, motCentral: str, mot2: str, listeMots: list[str] = None, distance: str = 'cosine') -> int:

    if listeMots is None:
        listeMots = modele.index_to_key

    listeMots = listeMots.copy()
    listeMots.remove(motCentral)
    distMots: list[float] = []
    pos = 1

    if distance == 'cosine':
        distMots = modele.distances(motCentral, listeMots)
        temp = sorted(distMots)
        pos = [temp.index(distMots[i]) for i in range(len(distMots)) if listeMots[i] == mot2][0] + 1
        #pos = modele.rank(motCentral, mot2)
    
    return pos, pos / len(listeMots)

In [13]:
print(get_position_proximite(w2vecR, "vie", "chocolat", list(vocabulaireCommun)))
print(get_position_proximite(w2vec.wv,  "vie", "chocolat", list(vocabulaireCommun)))

(3304, 0.857736240913811)
(2284, 0.592938733125649)


### Validation des proximités sous BATS

In [14]:
get_stats_comparaisons_BATS(ev, w2vecR)

{'taille_vocab': 3853,
 'nb_comp': 179,
 'rmse_freq': 0.38301971232118665,
 'err_moy_freq': 0.23272048451671376,
 'err_dis_cos': -0.6540805354466759,
 'rmse_dis_cos': 0.6860216614593655}

### Moyenne des similarités entre chaque mot du vocabulaire des deux modèles

In [16]:
def stats_comparaisons_intra_vocab(modele: models.KeyedVectors, reference: models.KeyedVectors):

    vocabulaire = list(set(modele.index_to_key).intersection(reference.index_to_key))
    stats = {}
    stats['taille_vocab'] = len(vocabulaire)
    stats['err_dis_cos'] = 0
    n = 0
    for mot1, mot2 in itertools.product(vocabulaire, vocabulaire):
        n += 1
        if mot1 == mot2:
            continue
        
        # Distance cosinus
        disMod = modele.distance(mot1, mot2)
        disRef = modele.distance(mot1, mot2)
        stats['err_dis_cos'] += disMod - disRef
    
    stats['err_dis_cos'] /= n
    return stats

In [17]:
stats_comparaisons_intra_vocab(ev, w2vecR)

# Tuning

Hyperparamètres considérés : 
- Type de modèle (CBOW, Skip-Gram) ;
- Taille de l'espace vectoriel (100 à 300) ;
- Taille de la fenêtre de contexte (2 à 6) ;
- Fréquence minimale des mots que l'on doit considérer (1, 2) ;

In [25]:
def tuning(documents : list[str], modeleReference: models.KeyedVectors):

    # Grille des hyperparamètres
    typesModele = [0, 1]
    taillesEV = range(100, 301, 50)
    taillesFenetre = range(2, 7)
    frequencesMin = [1, 2]

    nbModeles: int = len(typesModele * len(taillesEV) * len(taillesFenetre) * len(frequencesMin))
    print(f"Nombre de modèles à tester : {nbModeles}")
    print("0 % | ", end = '')
    tableauModeles = pd.DataFrame(columns = ["type_modele", "dimension_ev", "taille_fenetre", "frequence_min",
    "err_dis_cos", 'rmse_dis_cos', 'err_moy_freq', 'rmse_freq'])

    modelesFaits = 0
    score = np.inf
    bestModele: models.Word2Vec = None
    meilleursHP = {}
    # Réalisation d'un modèle pour chaque point de la grille
    for typeModele, N, fenetre, frequenceMin in itertools.product(typesModele, taillesEV, taillesFenetre, frequencesMin):

        modele = models.Word2Vec(sentences = documents,  vector_size = N, window = fenetre, min_count = frequenceMin, workers = 6, sg = typeModele)
        stats = get_stats_comparaisons_BATS(modele.wv, modeleReference)

        tableauModeles.loc[len(tableauModeles.index)] = ["Skip-Gram" if typeModele else "CBOW", N,
        fenetre, frequenceMin, stats["err_dis_cos"], stats["rmse_dis_cos"], stats['err_moy_freq'], stats['rmse_freq']]
        
        modelesFaits += 1
        if (percent := round(modelesFaits * 100 / nbModeles)) % 5 == 0 and round((modelesFaits -1) * 100 / nbModeles) != 0:
            print(f"{percent} % |", end = " ")

    return tableauModeles

In [26]:
if False:
    tableauModeles = tuning(docs, w2vecR)
    tableauModeles
    minRMSEDisCos = np.min(tableauModeles['rmse_dis_cos'])
    tableauModeles.loc[tableauModeles['rmse_dis_cos'] == minRMSEDisCos, :]

# Espace de documents

## 1) Valeur moyenne

La fonction suivante transforme un document en vecteurs en faisant la moyenne de la représentation vectorielle de ses mots.

In [27]:
def wvs_to_doc(doc: list[str], w2v: Callable, method: str = 'mean') -> np.ndarray:

    if method == 'mean':

        mean = np.zeros(len(w2v(doc[0])))
        for mot in doc:
            mean += w2v(mot)
        mean /= len(doc)

        return mean

In [28]:
toVec = lambda mot: ev[mot]

meanDocs = []
for doc in docs:
    meanDocs.append(wvs_to_doc(doc, toVec))
meanDocs = np.array(meanDocs)

## 2) Distance WMD 

In [33]:
# Prend dans notre cas 3h45min environ de calcul
if False:
    distance_wmd_tous_docs(docs, ev, retour = 'fichier', nomFichier = 'wmd/w2vec.txt', toInteger = True)

In [ ]:
distances = wmd.lecture_fichier_distances_wmd('../data/wmd/w2vec.txt', integer = True)

In [38]:
groupeDocs, positionCentres, nbIter, nbDistances, intraVar = k_med.k_medoides_wmd(docs, ev, distancesDocs = distances, graine = 1, nbCycles = 2)
groupeDocs

Cycle 1 :
Itération 1 : Création des groupes - Correction des tailles de groupe - part dans chaque groupe (%) : 5 5 80 5 5 (784 s) | Recherche du point moyen pour chaque groupe ( 0 1 2 3 4 ) - Stabilité des centres : X X X X X (194 s)
Itération 2 : Création des groupes - Correction des tailles de groupe - part dans chaque groupe (%) : 6 10 39 5 39 (75 s) | Recherche du point moyen pour chaque groupe ( 0 1 2 3 4 ) - Stabilité des centres : V V X V V (97 s)
Itération 3 : Création des groupes - Correction des tailles de groupe - part dans chaque groupe (%) : 6 9 61 5 19 (74 s) | Recherche du point moyen pour chaque groupe ( 0 1 2 3 4 ) - Stabilité des centres : V V V V V (128 s)
Cycle 2 :
Itération 1 : Création des groupes - Correction des tailles de groupe - part dans chaque groupe (%) : 80 5 5 5 5 (924 s) | Recherche du point moyen pour chaque groupe ( 0 1 2 3 4 ) - Stabilité des centres : X X X X X (205 s)
Itération 2 : Création des groupes - Correction des tailles de groupe - part dan

0       0
1       0
2       0
3       4
4       4
       ..
9496    2
9497    2
9498    1
9499    2
9500    2
Name: 2, Length: 9501, dtype: int64

In [39]:
nbIter

9

Note : Sans donner une taille minimale on obtient quelque chose de type 97 0 3 0 0. Prend moins de temps à calculer par contre

Récupération du vocabulaire et de la fréquence des mots dans chaque groupe de documents :

In [42]:
numerosGroupes = np.unique(groupeDocs)
vocabGroupes = []
for k in numerosGroupes:
    vocabGroupes.append({})
    for doc in [docs[i] for i in np.arange(len(docs))[groupeDocs == k]]:
        mots, comptage = np.unique(doc, return_counts=True)
        for i, mot in enumerate(mots):
            if mot in vocabGroupes[-1]:
                vocabGroupes[-1][mot] += comptage[i]
            else:
                vocabGroupes[-1][mot] = comptage[i]

    vocabGroupes[-1] = {key: value for key, value in sorted(vocabGroupes[-1].items(), reverse = True, key=lambda item: item[1])}

Affichage du mot le plus utilisé dans chaque groupe :

Ligne i : mot le plus utilisé dans le groupe i suivi du nombre de fois où on le retrouve dans le groupe 0, 1, ... etc.

In [46]:
for d in vocabGroupes:
    motPlusUtilise = list(d.keys())[0]
    print(motPlusUtilise, end = ' : ')
    for dPrim in vocabGroupes:
        print(dPrim[motPlusUtilise] if motPlusUtilise in dPrim else '0', end = ', ')
    print('\n')

transport : 442, 1, 18, 0, 11, 

plastique : 0, 357, 0, 305, 1, 

ville : 47, 2, 369, 1, 0, 

plastique : 0, 357, 0, 305, 1, 

produit : 10, 66, 9, 43, 689, 



Ligne i : Mot le plus utilisé dans le groupe i et ayant le plus d'importance par rapport aux autres groupes

In [64]:
motsDisc: list[str] = [''] * len(vocabGroupes)
quantiteMotsDisct: dict[list[int]] = {}
for k, d in enumerate(vocabGroupes):
    motPlusUtilise: str = None 
    freqMotPlusUtilise: float = 0
    quantiteMotsDisct[k] = [0] * len(vocabGroupes)
    

    for mot in d:
        temp: list[int] = []
        for dPrim in vocabGroupes:
            temp.append(dPrim[mot] if mot in dPrim else 0)
        if d[mot] == max(temp) and (f := d[mot] / np.sum(temp) > freqMotPlusUtilise):
            freqMotPlusUtilise = f
            motPlusUtilise = mot
            quantiteMotsDisct[k] = temp
    
    motsDisc[k] = motPlusUtilise
    print(motPlusUtilise, quantiteMotsDisct[k])

    """
    variationMotPlusUtilise: float = 0
    for mot in d:
        temp: list[int] = []
        for dPrim in vocabGroupes:
            temp.append(dPrim[mot] if mot in dPrim else 0)
        temp = np.array(temp, dtype = np.float64)
        if np.max(temp) == temp[k]:
            n = np.sum(temp)
            temp /= n
            norme = np.linalg.norm(temp - np.max(temp), 2) ** 2
            if norme > variationMotPlusUtilise:
                variationMotPlusUtilise = norme
                motPlusUtilise = mot
    
    motsDisc[k] = motPlusUtilise
    print(k, temp, motPlusUtilise)
    """

transport [442, 1, 18, 0, 11]
plastique [0, 357, 0, 305, 1]
ville [47, 2, 369, 1, 0]
bouteille [0, 29, 0, 222, 0]
produit [10, 66, 9, 43, 689]
